# Download de arquivos

In [3]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]
FOLDER_ID = '1NWtp0sb_DXDPXnUdNCal1rzoZw6zMjfc'


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("drive", "v3", credentials=creds)

        page_token = None
        # Call the Drive v3 API
        while True:
            # Call the Drive v3 API
            results = service.files().list(
                q=f"'{FOLDER_ID}' in parents",
                pageSize=10, fields="nextPageToken, files(id, name)",
                pageToken=page_token).execute()
            items = results.get('files', [])
    
            if not items:
                print('No files found.')
            else:
                for item in items:
                    print(u'{0} ({1})'.format(item['name'], item['id']))
    
    
                    file_id = item['id']
                    request = service.files().get_media(fileId=file_id)
    
                    with open('data/'+item['name'], 'wb') as fh:
                        downloader = MediaIoBaseDownload(fh, request)
                        done = False
                        while done is False:
                            status, done = downloader.next_chunk()
                            print("Download %d%%." % int(status.progress() * 100))
    
            page_token = results.get('nextPageToken', None)
            if page_token is None:
                break
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f"An error occurred: {error}")

    if __name__ == "__main__":
        main()

In [ ]:
main()